In [3]:
#Ecopy installation to python enviroment doesn't work with new pip versions
#To make it work, you need to install an older version of pip by "pip install pip==21.3.1"
#After that you can update your pip back by "pip install --upgrade pip"

from scipy.special import comb
import pandas as pd
import numpy as np
import scipy.misc
scipy.misc.comb = comb
import h3

import sys
sys.path.append('../')
from helper_functions.gridding import h3_grid

In [12]:
# Read and concatnate data into a single dataframe
df= pd.DataFrame({})
df = pd.read_csv('../../data/samples/tree_data_sample.csv')
  

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115722 entries, 0 to 115721
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Date            114067 non-null  object 
 1   Latitude        113082 non-null  float64
 2   Longitude       113082 non-null  float64
 3   Municipality    108751 non-null  object 
 4   TaxonID         115601 non-null  object 
 5   ScientificName  115601 non-null  object 
 6   verbatim        115651 non-null  object 
dtypes: float64(2), object(5)
memory usage: 6.2+ MB


In [17]:
#Doing the gridding
#Drop NAs from Lat and Lon. Otherwise h3 cannot work with these.
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)
df.head()
grid_object = h3_grid()
grid_object.fit(df)
grid_object.grid_info()
grid = grid_object.grid_info()

In [23]:
grid = grid.drop(columns=['observations_id', 'neighbors'])

In [25]:
grid = grid.rename(columns={'count':'count_1'})

In [26]:
grid['count_2'] = grid['count_1']

In [31]:
grid = grid.drop(columns=['h3_cell'])

In [32]:
grid.corr(method='pearson')

,count_1,count_2
count_1,1.0,1.0
count_2,1.0,1.0
